# Combinar xlsx de export comments
Script para juntar los archivos de export comments en uno solo, incluyendo el id del post al que corresponde. 

In [1]:
# Requerimientos
# pip install openpyxl

# Importar
import glob
import openpyxl
from pathlib import Path
import pandas as pd

/Users/andres/dev/python/facebook_scraping/env/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# Path a la carpeta con archivos xlsx a procesar
path_dir = 'MargaritaMorenoColima'
# Nombre del archivo de salida
file_name = 'DS Margarita Moreno.xlsx'

## Entendiendo la estructura del archivo

In [3]:
# Obtener los archivos xlsx
xlsx_files = [path for path in Path(path_dir).rglob('*.xlsx')]
# Para purebas seleccionar solo el primer archivo
xlsx_file = xlsx_files[0]
# Instanciar objeto y activar unica hoja del xlsx
wb_obj = openpyxl.load_workbook(xlsx_file)
sheet = wb_obj.active

In [4]:
# El id del post está en el link que se ubica en B2
print(sheet['B2'].value.split('/')[-1].split('?')[0])

141491397977758


In [5]:
# Desplegar los primeros 3 registros del xlsx
# Se muestran desde el renglón 7, antes son espacios en blanco y link
for row in sheet.iter_rows(min_row=7, max_row=10, min_col=3, max_col=9):
    for cell in row:
        print(cell.value, end=" ")
    print()

Lola Marquez ID: 1273157993 2021-04-13 13:37:39 2 None Buenos días
[PHOTO] https://external.fman4-2.fna.fbcdn.net/safe_image.php?d=AQHp5Z3hfzYJm8P6&w=396&h=396&url=https%3A%2F%2Fmedia0.giphy.com%2Fmedia%2Fv1.Y2lkPTEyMGMwMTQ3aWxxdWVmMzk2Y3JjYzYwaHBzODJzM2dtc2FpejZuZnRpNmVxb3Vvaw%2FLb4IZLmCfALhm%2Fgiphy.gif&ext=jpg&_nc_oe=6ef14&_nc_sid=220845&ccb=3-5&_nc_hash=AQE7NocOOFHj8Y7f view comment 
Mayra Alejandra Evangelista Leal ID: 100000111377694 2021-04-13 13:55:37 1 None Con todo Margarita Moreno  ❤💙💛 view comment 
Efrain Medina Soto ID: 100009900467169 2021-04-13 14:51:09 1 None UN ABRAZO MARGARITA view comment 
Andrea Cortes ID: 100063630430132 2021-04-13 15:47:46 1 None Desde el cielo ella la guiara . Un abrazo con aprecio view comment 


In [6]:
# Verificar los renglones (el ultimo renglón no tiene valores) y columnas
print(sheet.max_row, sheet.max_column)

24 9


In [7]:
# Obtener link en una celda
comment_link = sheet['I7'].hyperlink.target
print(comment_link)
# Limpiar link para sólo conservar el id
comment_id = comment_link.split('/')[-1]
print(comment_id)

https://www.exportcomments.com/done/9c93477a-cc5d-4dcb-b0b9-a7a03a8a9a82/preview/141495294644035
141495294644035


## Juntando archivos
Consideraciones
- No tener abierto los archivos
- Solo tener archivos exportado por export comments en la carpeta

In [8]:
def get_post_id(sheet):
    return [sheet['B2'].value.split('/')[-1].split('?')[0]]

In [9]:
def get_data_row(row):
    user_name = row[0].value
    profile_id = row[1].value.split(' ')[-1]
    date_time = row[2].value
    likes = row[3].value
    comments = row[5].value
    # Obtener link del comentario
    comment_link = row[6].hyperlink.target
    # Limpiar y guaardar sólo id
    comment_id = comment_link.split('/')[-1]
    return [user_name, profile_id, date_time, likes, comments, comment_id]

In [10]:
# Nombres de columnas
columns = ['post_id','user_name','profile_id','date_time','likes','comments','comment_id']
data = []

# Obteniendo todos los archivos xlsx
xlsx_files = [path for path in Path(path_dir).rglob('*.xlsx')]
# Generando objetos para cada archivo
wb_objects = [openpyxl.load_workbook(wb) for wb in xlsx_files]

# Procesando todos los objetos xlsx
for wb_obj in wb_objects:
    sheet = wb_obj.active
    post_id = get_post_id(sheet)
    # El maximo renglón a procesar es -1 ya que el último es de valores nulos
    max_row = sheet.max_row - 1
    # Procesado de todos los valores con función get_data_row y get_post_id
    for row in sheet.iter_rows(min_row=7, max_row=max_row, min_col=3, max_col=9):
        comment_data = post_id + get_data_row(row)
        # El rengló se agrega a la lista
        data.append(comment_data)
        
# Con los datos y columnas se genera el dataframe
df = pd.DataFrame(data, columns=columns)

### Revisando datos del dataframe

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5535 entries, 0 to 5534
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   post_id     5535 non-null   object        
 1   user_name   5535 non-null   object        
 2   profile_id  5535 non-null   object        
 3   date_time   5535 non-null   datetime64[ns]
 4   likes       5535 non-null   int64         
 5   comments    5535 non-null   object        
 6   comment_id  5535 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 302.8+ KB


In [12]:
df.head(5)

,post_id,user_name,profile_id,date_time,likes,comments,comment_id
0,141491397977758,Lola Marquez,1273157993,2021-04-13 13:37:39,2,Buenos días\n[PHOTO] https://external.fman4-2....,141495294644035
1,141491397977758,Mayra Alejandra Evangelista Leal,100000111377694,2021-04-13 13:55:37,1,Con todo Margarita Moreno ❤💙💛,141508141309417
2,141491397977758,Efrain Medina Soto,100009900467169,2021-04-13 14:51:09,1,UN ABRAZO MARGARITA,141540954639469
3,141491397977758,Andrea Cortes,100063630430132,2021-04-13 15:47:46,1,Desde el cielo ella la guiara . Un abrazo con ...,141575194636045
4,141491397977758,La More Corona,100028791736347,2021-04-13 18:00:37,1,"Es la señora, Nora",141644971295734


### Guardando en archivo

In [13]:
df.to_excel(file_name, sheet_name='Comments', header=True, index=False)

## Referencias

- https://www.marsja.se/your-guide-to-reading-excel-xlsx-files-in-python/
- https://stackoverflow.com/questions/29792134/how-we-can-use-iter-rows-in-python-openpyxl-package
- https://openpyxl.readthedocs.io/en/stable/datetime.html
- https://newbedev.com/extracting-hyperlinks-from-excel-xlsx-with-python
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html